<a href="https://colab.research.google.com/github/Paul33333/-Generative-LLM-as-Verifiers-/blob/main/%E3%80%90Generative_LLM_as_Verifiers%E3%80%91%E6%8E%A8%E7%90%86%E5%8A%A0%E9%80%9F%E7%AF%87%EF%BC%9A%E6%97%A9%E5%81%9C%E6%B3%95%2B%E5%A4%8D%E7%94%A8KV%E7%BC%93%E5%AD%98%2B%E5%B9%B6%E8%A1%8C%E6%8E%A8%E7%90%86%EF%BC%8C%E5%AE%9E%E7%8E%B0%E6%8E%A8%E7%90%86%E6%95%88%E7%8E%87%E6%8F%90%E5%8D%87%E5%87%A0%E5%8D%81%E5%80%8D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import time
import torch
import numpy as np
from transformers import AutoModelForCausalLM, AutoTokenizer
from typing import Union, Optional, List, Tuple, Dict
device = "cuda" # the device to load the model onto

model_path = "Qwen/Qwen2-1.5B-Instruct"
qwen_model = AutoModelForCausalLM.from_pretrained(
    model_path,
    torch_dtype="auto",
    device_map="auto"
)
qwen_tokenizer = AutoTokenizer.from_pretrained(model_path)
qwen_model.eval()

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 1536)
    (layers): ModuleList(
      (0-27): 28 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear(in_features=1536, out_features=1536, bias=True)
          (k_proj): Linear(in_features=1536, out_features=256, bias=True)
          (v_proj): Linear(in_features=1536, out_features=256, bias=True)
          (o_proj): Linear(in_features=1536, out_features=1536, bias=False)
          (rotary_emb): Qwen2RotaryEmbedding()
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=1536, out_features=8960, bias=False)
          (up_proj): Linear(in_features=1536, out_features=8960, bias=False)
          (down_proj): Linear(in_features=8960, out_features=1536, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm()
        (post_attention_layernorm): Qwen2RMSNorm()
      )
    )
    (norm): Qwen2RMSNorm()
  )
  (lm_head): Linear(in_

In [ ]:
!nvidia-smi

Fri Nov  8 16:13:47 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.161.08             Driver Version: 535.161.08   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A30                     On  | 00000000:CE:00.0 Off |                   On |
| N/A   31C    P0              54W / 165W |                  N/A |     N/A      Default |
|                                         |                      |              Enabled |
+-----------------------------------------+----------------------+--

In [ ]:
article = """下面是两则新闻快讯：
1）日本原子弹氢弹受害者团体协议会获得诺贝尔和平奖后，以色列驻日本大使科恩在社交媒体X上用日语发表声明，他首先祝贺日本原子弹氢弹受害者团体协议会获得诺贝尔和平奖，称其“追求和平与正义的功绩得到了认可”，但他进一步指出，加沙被伊斯兰组织哈马斯所控制，并指责哈马斯是“将自己公民当作人肉盾牌，犯下双重战争罪的恐怖组织”。
对于箕牧提到“加沙在人们怀中的孩子们正在流着很多血，这让我想起80年前的日本”，“加沙冲突中受伤的孩子们，与原爆孤儿的形象重叠”这种说法，科恩反驳称“将加沙与80年前的日本进行比较是不恰当且缺乏依据的”。
科恩指出：在去年10月7日哈马斯的大屠杀中，有1200人遇害，251人被绑架，但未见日本原子弹氢弹受害者团体协议会代表委员对此发表声明。这样的比较扭曲了历史，并贬低了遭恐怖袭击的受害者。

2）在北京时间3月3日的NBA常规赛中，洛杉矶湖人队与丹佛掘金队的比赛成为了篮球历史上的一个重要时刻。这场比赛不仅是两队之间的较量，更是勒布朗·詹姆斯个人职业生涯的一个里程碑。与此同时，国际局势也发生了重大变化，俄乌冲突的持续影响着全球政治经济格局。
当比赛进行到第二节，詹姆斯凭借一记精彩的得分，得到本场比赛第10分，职业生涯总得分超过40000分，成为NBA历史上首位达到这一成就的球员。这一壮举不仅在篮球界引起了巨大的轰动，也在全球范围内引起了广泛关注。然而，就在这一天，俄乌冲突也进入了一个新的阶段，俄罗斯总统普京表示，部分西方国家的行为加剧并延长了俄乌冲突。
这场冲突不仅对欧洲安全结构与欧亚地缘政治有深刻的影响，也将在一定程度上重塑全球地缘政治格局。
俄乌冲突的爆发，标志着欧亚地缘政治重新走向冷战的回归。这场冲突将带来欧洲安全格局与结构的根本性改变，使得俄罗斯的安全关切长期得不到重视。
同时，俄乌冲突的持续也给全球地缘政治带来深刻的影响，其中俄美关系将发生自冷战结束以来最本质性的变化。
在这样一个充满紧张和不确定性的国际背景下，詹姆斯的成就显得尤为珍贵，它提醒着人们，即使在动荡不安的时代，人类依然能够通过体育和个人努力实现伟大的成就。而俄乌冲突的持续，也让我们更加深刻地意识到和平的宝贵和国际合作的重要性。在全球范围内，无论是体育竞技还是国际政治，我们都期待能够有更多的对话和理解，以促进和平与发展。"""

prompt = f"""```
{article}
```

假设你是一个专业的文本分类器，请根据上文判断这篇文章最适合被划归到以下哪个类目结果，请直接输出类目结果，不要解释：
财经，政治，娱乐，体育，军事，科技，农业民生，能源，汽车，创新设计，环境，其它"""

answer_list = '财经，政治，娱乐，体育，军事，科技，农业民生，能源，汽车，创新设计，环境，其它'.split('，')

temperature = 1

In [ ]:
answer_list

['财经', '政治', '娱乐', '体育', '军事', '科技', '农业民生', '能源', '汽车', '创新设计', '环境', '其它']

In [ ]:
print(prompt)

```
下面是两则新闻快讯：
1）日本原子弹氢弹受害者团体协议会获得诺贝尔和平奖后，以色列驻日本大使科恩在社交媒体X上用日语发表声明，他首先祝贺日本原子弹氢弹受害者团体协议会获得诺贝尔和平奖，称其“追求和平与正义的功绩得到了认可”，但他进一步指出，加沙被伊斯兰组织哈马斯所控制，并指责哈马斯是“将自己公民当作人肉盾牌，犯下双重战争罪的恐怖组织”。
对于箕牧提到“加沙在人们怀中的孩子们正在流着很多血，这让我想起80年前的日本”，“加沙冲突中受伤的孩子们，与原爆孤儿的形象重叠”这种说法，科恩反驳称“将加沙与80年前的日本进行比较是不恰当且缺乏依据的”。
科恩指出：在去年10月7日哈马斯的大屠杀中，有1200人遇害，251人被绑架，但未见日本原子弹氢弹受害者团体协议会代表委员对此发表声明。这样的比较扭曲了历史，并贬低了遭恐怖袭击的受害者。

2）在北京时间3月3日的NBA常规赛中，洛杉矶湖人队与丹佛掘金队的比赛成为了篮球历史上的一个重要时刻。这场比赛不仅是两队之间的较量，更是勒布朗·詹姆斯个人职业生涯的一个里程碑。与此同时，国际局势也发生了重大变化，俄乌冲突的持续影响着全球政治经济格局。
当比赛进行到第二节，詹姆斯凭借一记精彩的得分，得到本场比赛第10分，职业生涯总得分超过40000分，成为NBA历史上首位达到这一成就的球员。这一壮举不仅在篮球界引起了巨大的轰动，也在全球范围内引起了广泛关注。然而，就在这一天，俄乌冲突也进入了一个新的阶段，俄罗斯总统普京表示，部分西方国家的行为加剧并延长了俄乌冲突。
这场冲突不仅对欧洲安全结构与欧亚地缘政治有深刻的影响，也将在一定程度上重塑全球地缘政治格局。
俄乌冲突的爆发，标志着欧亚地缘政治重新走向冷战的回归。这场冲突将带来欧洲安全格局与结构的根本性改变，使得俄罗斯的安全关切长期得不到重视。
同时，俄乌冲突的持续也给全球地缘政治带来深刻的影响，其中俄美关系将发生自冷战结束以来最本质性的变化。
在这样一个充满紧张和不确定性的国际背景下，詹姆斯的成就显得尤为珍贵，它提醒着人们，即使在动荡不安的时代，人类依然能够通过体育和个人努力实现伟大的成就。而俄乌冲突的持续，也让我们更加深刻地意识到和平的宝贵和国际合作的重要性。在全球范围内，无论是体育竞技还是国际政治，我们都期待能够有更多的对话和理解，以促进和平与发展。
```

假设你是一个专业的文本分

In [ ]:
class answer_speculative_decoding:
    """
    版本3： 统一各计算方式中的计算口径：一致忽略对齐聊天模版时answer尾缀后补的'<im_end>'和'\n'这两个token，不将其token's prob纳入考虑
    """
    def __init__(self, model, tokenizer):
        self.model = model
        self.tokenizer = tokenizer
        self.device = "cuda"

    def softmax(self, x):
        """Compute softmax values for each sets of scores in x."""
        return np.exp(x) / np.sum(np.exp(x), axis=0)

    def empty_cache(self):
        torch.cuda.empty_cache()
        torch.cuda.ipc_collect()

    def build_conversations(self, prompt:str, answer:Optional[str]=None, system_prompt:str="You are a helpful assistant", append_answer:bool=True) -> list:
        out = [{"role": "system", "content": system_prompt},{"role": "user", "content": prompt}]
        if append_answer:
            if answer is None:
                raise ValueError("`answer` cannot be None when `append_answer` is True.")
            out.append({"role": "assistant", "content": answer})
        return out

    def pad_operation(self, x:torch.tensor, max_dim:int, value:int):
        return torch.nn.functional.pad(x, (0, max_dim - x.shape[1]), mode='constant', value=value)

    # 用于KV缓存复制：为了批量推理，复用多次question_past_kv_cache以并行推理，需要我们扩充kv_cache以对齐形状，具体通过expand函数实现，注意因为question_past_kv_cache为元组数据(不可变的数据结构据)，不便直接更改，此处选择新建一个kv缓存，注意这并不会增加显存占用
    def kv_cache_expand(self, expand_num: int, past_kv_cache: Tuple[Tuple[torch.Tensor, torch.Tensor], ...]) -> Tuple[Tuple[torch.Tensor, torch.Tensor], ...]:
        """
        扩展 KV 缓存以支持批量推理。

        参数:
        - expand_num: 扩展的批量大小倍数。
        - past_kv_cache: 输入的 KV 缓存，是一个元组，其中每个元素是一个包含两个张量的元组（分别表示 Key 和 Value）。

        返回:
        - 扩展后的 KV 缓存，形状为 (batch_size * expand_num, num_heads, sequence_length, head_size)。
        """
        if not past_kv_cache:
            raise ValueError("past_kv_cache 不能为空")

        # 获取 KV 缓存的形状信息
        batch_size = past_kv_cache[0][0].shape[0]
        num_heads = past_kv_cache[0][0].shape[1]
        sequence_length = past_kv_cache[0][0].shape[2]
        head_size = past_kv_cache[0][0].shape[3]

        # 创建新的 KV 缓存
        expanded_kv_cache = []
        for layer_kv_tuple in past_kv_cache:
            if len(layer_kv_tuple) != 2:
                raise ValueError("每个层的 KV 缓存应包含两个张量（Key 和 Value）")

            key_tensor, value_tensor = layer_kv_tuple
            if key_tensor.shape != value_tensor.shape:
                raise ValueError("Key 和 Value 张量的形状应一致")

            expanded_key = key_tensor.expand(batch_size * expand_num, num_heads, sequence_length, head_size)
            expanded_value = value_tensor.expand(batch_size * expand_num, num_heads, sequence_length, head_size)
            expanded_kv_cache.append((expanded_key, expanded_value))

        return tuple(expanded_kv_cache)

    # 返回标签的概率及最大标签等信息
    def return_probability(self, label_list:List[str], probability_list:List[float]):
        return_dict = {}
        return_dict['label_prob'] = sorted({label:prob for label, prob in zip(label_list, probability_list)}.items(), key=lambda x: x[1], reverse=True)
        return_dict['label_prob_normalized'] = sorted({label:prob for label, prob in zip(label_list, self.softmax(probability_list))}.items(), key=lambda x: x[1], reverse=True)
        return_dict['label'] = return_dict['label_prob'][0][0]
        return_dict['probability'] = return_dict['label_prob'][0][1]
        return_dict['probability_normalized'] = return_dict['label_prob_normalized'][0][1]

        return return_dict

    def return_conversations_inputids(self, prompt:str, answer_list:List[str]):
        conversations = list(map(lambda x: self.build_conversations(prompt, x), answer_list))
        conversations_template = list(map(lambda x: self.tokenizer.apply_chat_template(x, tokenize=False, add_generation_prompt=False), conversations))
        conversations_inputids = list(map(lambda x: self.tokenizer([x], return_tensors="pt").input_ids.to(self.device), conversations_template))

        return conversations_inputids

    # 串行解码，目标标签one by one 地执行model推理，显存占用小，以推理时间换推理空间（显存），耗时偏长，推理的时间复杂度o(n)
    def answer_decode(self, prompt:str, temperature:float, answer_list:list, geometric_mean:bool=False):
        conversations_inputids = self.return_conversations_inputids(prompt, answer_list)
        #忽略最后的im_end和\n
        conversations_inputids = [i[:,:-2] for i in conversations_inputids]

        with torch.no_grad():
            answers_scorelist = []
            for i in range(len(answer_list)):
              assistant_token_index = torch.where(conversations_inputids[i] == 77091)[1] # 77091 -> model.generation_config.assistant_token_id
              # ASSISTANT后面的\n坐标位置开始切片logit，并忽略最后的im_end和\n
              answer_logit = self.model(conversations_inputids[i]).logits[0, assistant_token_index+1:-1, :]
              labels = conversations_inputids[i][0, assistant_token_index+2:]
              # 计算基于语言模型的回答部分的概率输出: p1*p2*...，如果设置为几何平均数口径，还需要开n次方
              if geometric_mean:
                    answer_score = (((torch.gather(torch.nn.functional.softmax(answer_logit/temperature, dim=-1), dim=-1, index=labels.unsqueeze(-1)).squeeze(-1)).prod(dim=-1))**(1/labels.shape[-1])).item()
              else:
                    answer_score = torch.gather(torch.nn.functional.softmax(answer_logit/temperature, dim=-1), dim=-1, index=labels.unsqueeze(-1)).squeeze(-1).prod(dim=-1).item()
              answers_scorelist.append(answer_score)

        return self.return_probability(answer_list, answers_scorelist)

        self.empty_cache()

    # 并行解码，目标标签并行执行model推理（batch size = 标签num），显存占用大，以推理空间（显存）换推理时间，推理速度变快, 推理的时间复杂度o(1)
    def answer_batch_decode(self, prompt:str, temperature:float, answer_list:list, geometric_mean:bool=False):
        conversations_inputids = self.return_conversations_inputids(prompt, answer_list)
        #忽略最后的im_end和\n
        conversations_inputids = [i[:,:-2] for i in conversations_inputids]

        # 对话数据填充对齐长度
        max_dim = max([i.size()[-1] for i in conversations_inputids])
        conversations_inputids_padded = torch.concat(list(map(lambda x: self.pad_operation(x, max_dim, self.model.generation_config.eos_token_id[-1]), conversations_inputids)))

        assistant_token_index = torch.where(conversations_inputids_padded == 77091)[1] # 77091 -> model.generation_config.assistant_token_id
        #确保batch_size中的每行tensor中的assistant_token所在的index都一致
        assert torch.unique(assistant_token_index).shape[0] == 1

        with torch.no_grad():
            # size(): batch_size, padded_sequence_length, vocabulary_size
            answers_logit = self.model(conversations_inputids_padded).logits
            # ASSISTANT后面的\n坐标位置开始切片logit
            answer_logits = answers_logit[:, torch.unique(assistant_token_index)+1:-1, :]
            # ASSISTANT后面的第二个token及以后为label token
            labels = conversations_inputids_padded[:, torch.unique(assistant_token_index)+2:]
            # 掩码掉填充token后，计算基于语言模型的回答部分的概率输出: p1*p2*...，如果设置为几何平均数口径，还需要开n次方
            if geometric_mean:
                label_len = torch.where(labels!=self.model.generation_config.eos_token_id[-1], 1, 0).sum(dim=-1)
                answers_scorelist = (((torch.where(labels!=self.model.generation_config.eos_token_id[-1], torch.gather(torch.nn.functional.softmax(answer_logits/temperature, dim=-1), dim=-1, index=labels.unsqueeze(-1)).squeeze(-1), 1)).prod(dim=-1)) ** (1/label_len)).tolist()
            else:
                answers_scorelist = (torch.where(labels!=self.model.generation_config.eos_token_id[-1], torch.gather(torch.nn.functional.softmax(answer_logits/temperature, dim=-1), dim=-1, index=labels.unsqueeze(-1)).squeeze(-1), 1)).prod(dim=-1).tolist()

        return self.return_probability(answer_list, answers_scorelist)

        self.empty_cache()

    # 使用“早停法”（只考虑第一个预测输出的token的概率）判断预测标签及概率，大幅减少推理耗时,推理的时间复杂度o(1)
    # one task with multi candidate label
    def answer_decode_early_stopping_method(self, prompt:str, temperature:float, answer_list:list):
        question = self.build_conversations(prompt, append_answer=False)
        question_template = self.tokenizer.apply_chat_template(question, tokenize=False, add_generation_prompt=True)
        question_inputids = self.tokenizer([question_template], return_tensors="pt").input_ids.to(self.device)

        conversations_inputids = self.return_conversations_inputids(prompt, answer_list)

        # 对话数据填充对齐长度
        max_dim = max([i.size()[-1] for i in conversations_inputids])
        conversations_inputids_padded = torch.concat(list(map(lambda x: self.pad_operation(x, max_dim, self.model.generation_config.eos_token_id[-1]), conversations_inputids)))

        # 查看回答侧的第一个输出的token id
        first_token_in_answers = conversations_inputids_padded[:, question_inputids.shape[1]]
        # 检查回答侧的first_token是否完全不一致，完全不一致的话，使用“早停法”（只考虑第一个输出token的概率）判断预测标签及概率
        if torch.unique(first_token_in_answers).shape[0] == conversations_inputids_padded.shape[0]:

            with torch.no_grad():
                answer_first_token_logit = self.model(question_inputids).logits[0, -1]
                answer_first_token_probs_list = torch.gather(torch.nn.functional.softmax(answer_first_token_logit/temperature, dim=-1), dim=-1, index=first_token_in_answers).tolist()

            return self.return_probability(answer_list, answer_first_token_probs_list)

        else:
            self.answer_decode(prompt, temperature, answer_list)

        self.empty_cache()

    # 复用KV缓存 -> 后串行推理
    # 时间复杂度：o(n+1)
    # 1次问题的推理，获得KV缓存 + n次目标标签的推理
    def answer_decode_using_KVcache(self, prompt:str, temperature:float, answer_list:list, geometric_mean:bool=False):
        # 构建聊天模板
        question = self.build_conversations(prompt, append_answer=False)
        question_template = self.tokenizer.apply_chat_template(question, tokenize=False, add_generation_prompt=True)
        question_inputids = self.tokenizer(question_template, return_tensors="pt")['input_ids'].to(self.device)

        # 先获得问题部分的past_key_vlaues(重复利用KV缓存)
        with torch.no_grad():
            question_past_kv_cache = self.model(question_inputids[:, :-1]).past_key_values #留意question_inputids[:,:-1]的操作

        # 然后串行依次推理标签概率，推理时，复用KV缓存的同时，每次inputids需要填写新加入的tokens即可（即标签）
        answers_scorelist = []

        with torch.no_grad():
            for answer in answer_list:
                # 构建目标tokens
                new_tokens = self.tokenizer('\n' + answer, return_tensors='pt').input_ids.to(self.device)
                answer_logits = self.model(new_tokens, past_key_values=question_past_kv_cache).logits
                # 计算基于语言模型的回答部分的概率输出: p1*p2*...，如果设置为几何平均数口径，还需要开n次方
                if geometric_mean:
                      answer_score = ((torch.gather(torch.nn.functional.softmax(answer_logits[:, :-1, :]/temperature, dim=-1), index=new_tokens[:, 1:].unsqueeze(-1), dim=-1).squeeze(-1).prod(dim=-1))**(1/(new_tokens.shape[-1]-1))).item()
                else:
                      answer_score = torch.gather(torch.nn.functional.softmax(answer_logits[:, :-1, :]/temperature, dim=-1), index= new_tokens[:, 1:].unsqueeze(-1), dim=-1).squeeze(-1).prod(dim=-1).item()
                answers_scorelist.append(answer_score)

        return self.return_probability(answer_list, answers_scorelist)

        self.empty_cache()

    # 复用KV缓存 -> 后并行推理
    # 时间复杂度：o(2)
    # 1次问题的推理获得KV缓存 + 1次目标标签的推理
    # 但会增强一部分空间复杂度（显存增加），但对比不用kv缓存的并行推理模型，显存增加小很多
    def answer_batch_decode_using_KVcache(self, prompt:str, temperature:float, answer_list:list, geometric_mean:bool=False):
        # 构建聊天模版
        question = self.build_conversations(prompt, append_answer=False)
        question_template = self.tokenizer.apply_chat_template(question, tokenize=False, add_generation_prompt=True)
        question_inputids = self.tokenizer(question_template, return_tensors="pt")['input_ids'].to(self.device)

        # 先获得问题部分的past_key_vlaues(重复利用KV缓存)
        with torch.no_grad():
            question_past_kv_cache = self.model(question_inputids[:, :-1]).past_key_values #留意question_inputids[:,:-1]的操作

            #然后为了batch并行推理，对kvcache进行复制expand
            question_past_kv_cache_expand = self.kv_cache_expand(len(answer_list), question_past_kv_cache)

        # 然后并行一次性推理标签概率，推理时，需要把标签的tokens对齐（填充），具体如下：
        answer_inputids = list(map(lambda x: self.tokenizer('\n' + x, return_tensors="pt").input_ids.to(self.device), answer_list)) #注意前缀\n
        # 对话数据填充对齐长度
        max_dim = max([i.size()[-1] for i in answer_inputids])
        answer_inputids_padded = torch.concat(list(map(lambda x: self.pad_operation(x, max_dim, self.model.generation_config.eos_token_id[-1]), answer_inputids)))

        # 推理标签的概率
        with torch.no_grad():
            batch_answers_logits = self.model(answer_inputids_padded, past_key_values=question_past_kv_cache_expand).logits
            # 掩码掉填充token后，计算基于语言模型的回答部分的概率输出: p1*p2*...，如果设置为几何平均数口径，还需要开n次方
            if geometric_mean:
                label_len = torch.where(answer_inputids_padded[:, 1:]!=self.model.generation_config.eos_token_id[-1], 1.0, 0.0).sum(dim=-1)
                answers_scorelist = ((torch.where(answer_inputids_padded[:, 1:]!=self.model.generation_config.eos_token_id[-1], torch.gather(torch.nn.functional.softmax(batch_answers_logits[:,:-1]/temperature, dim=-1), dim=-1, index=answer_inputids_padded[:, 1:].unsqueeze(-1)).squeeze(-1), 1.0).prod(dim=-1))**(1/label_len)).tolist()
            else:
                answers_scorelist = torch.where(answer_inputids_padded[:, 1:]!=self.model.generation_config.eos_token_id[-1], torch.gather(torch.nn.functional.softmax(batch_answers_logits[:,:-1]/temperature, dim=-1), dim=-1, index=answer_inputids_padded[:, 1:].unsqueeze(-1)).squeeze(-1), 1.0).prod(dim=-1).tolist()

        return self.return_probability(answer_list, answers_scorelist)

        self.empty_cache()

    ## 使用“早停法”+ "kv cache" + "batch inference" 应对 multi task parrellel inference任务
    # multi task with one article，整体时间复杂度o(2)
    def MuitiTask_batch_decode_using_KVcache_and_EarlyStop(self, task:Dict[str, List[str]], article:str, temperature:float):
        """
        task为dict字典数据类型，其中
        - key为每个任务task:str
        - value 为每个任务的多个候选标签：List[str]
        ------
        整体的时间复杂度为o(2)
        """
        article = self.build_conversations(f'```\n{article}\n```\n\n', append_answer=False) #文章用代码块包裹了（```\n{article}\n```）
        article_template = self.tokenizer.apply_chat_template(article, tokenize=False, add_generation_prompt=False)
        article_inputids = self.tokenizer([article_template], return_tensors="pt").input_ids[:, :-2].to(self.device) # 略去(排除)最后两个token的id，即em_end和\n
        # 先获得文章部分的past_key_vlaues(重复利用KV缓存)
        with torch.no_grad():
            article_past_kv_cache = self.model(article_inputids).past_key_values
            #然后为了Multi task的batch并行推理，对kv cache进行复制expand
            article_past_kv_cache_expand = self.kv_cache_expand(len(task.keys()), article_past_kv_cache)

        # 再获得每个task问题的对应的token id,记得尾缀后补'<|im_end|>\n<|im_start|>assistant\n'以对齐user问题部分的聊天模板
        questions_inputids = list(map(lambda x: self.tokenizer(x + '<|im_end|>\n<|im_start|>assistant\n', return_tensors="pt").input_ids.to(self.device), task.keys()))
        # question tokenid lens填充pad token对齐长度
        max_dim = max([i.size()[-1] for i in questions_inputids])
        questions_inputids_padded = torch.concat(list(map(lambda x: self.pad_operation(x, max_dim, self.model.generation_config.eos_token_id[-1]), questions_inputids)))

        # 然后使用早停法+并行推理，一次性地完成多个任务的多个标签的概率推理，具体如下：
        with torch.no_grad():
            batch_answers_logits = self.model(questions_inputids_padded, past_key_values=article_past_kv_cache_expand).logits

        MultiTask_result = {}

        for task_index in range(len(list(task.keys()))):
            ith_task_question, ith_task_answer_list =  list(task.keys())[task_index], list(task.values())[task_index]
            # 查看回答侧的第一个输出的token id
            first_token_in_answers = torch.tensor(list(map(lambda x: self.tokenizer(x)['input_ids'][0], ith_task_answer_list))).to(self.device)
            # 确保每个答案标签的first token都不一致
            assert torch.unique(first_token_in_answers).shape[0] == first_token_in_answers.shape[0], f"候选答案的标签存在first token相同的情况，需要确保每个候选答案的first token都不同，请检查你的候选答案：{ith_task_answer_list}"
            # 找到回答侧的第一个输出的token前一位的位置坐标
            first_answer_token_index = torch.where(questions_inputids_padded[task_index]==self.tokenizer.encode('\n')[0])[0][-1].item() #找到最后一个'\n'该token的元素位置
            answer_first_token_logit = batch_answers_logits[task_index, first_answer_token_index, :]
            # 使用“早停法”（只考虑第一个输出token的概率）判断预测标签及概率
            answer_first_token_probs_list = torch.gather(torch.nn.functional.softmax(answer_first_token_logit/temperature, dim=-1), dim=-1, index=first_token_in_answers).tolist()
            MultiTask_result[ith_task_question] = self.return_probability(ith_task_answer_list, answer_first_token_probs_list)

        return  MultiTask_result

        self.empty_cache()

In [ ]:
qwen_answer_decoding = answer_speculative_decoding(qwen_model, qwen_tokenizer)

### 原始的**串行推理**模式（不使用kv缓存）
- 低显存占用
- 耗时偏长 -> 时间换空间

In [ ]:
###
start = time.time()
response = qwen_answer_decoding.answer_decode(prompt, temperature, answer_list, geometric_mean=True)
end = time.time()
print(f'累计耗时{round(end-start,3)}秒')
response

累计耗时1.035秒


{'label_prob': [('体育', 0.9863552451133728),
  ('政治', 0.00753091461956501),
  ('财经', 0.002444930374622345),
  ('创新设计', 0.0016899178735911846),
  ('娱乐', 0.0008994395611807704),
  ('农业民生', 0.0005840228404849768),
  ('其它', 0.0002576937258709222),
  ('科技', 7.859215111238882e-05),
  ('能源', 6.12076255492866e-05),
  ('军事', 7.781676686136052e-06),
  ('汽车', 3.45309831573104e-06),
  ('环境', 8.201827199627587e-07)],
 'label_prob_normalized': [('体育', 0.19579674711270587),
  ('政治', 0.07357113682745486),
  ('财经', 0.07319790511451282),
  ('创新设计', 0.07314266063886124),
  ('娱乐', 0.07308486579770794),
  ('农业民生', 0.07306181724414622),
  ('其它', 0.07303797893579142),
  ('科技', 0.07302489889011018),
  ('能源', 0.07302362939792348),
  ('军事', 0.07301972814544845),
  ('汽车', 0.07301941207451665),
  ('环境', 0.07301921982082091)],
 'label': '体育',
 'probability': 0.9863552451133728,
 'probability_normalized': 0.19579674711270587}

In [ ]:
!nvidia-smi

Fri Nov  8 16:14:05 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.161.08             Driver Version: 535.161.08   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A30                     On  | 00000000:CE:00.0 Off |                   On |
| N/A   31C    P0              54W / 165W |                  N/A |     N/A      Default |
|                                         |                      |              Enabled |
+-----------------------------------------+----------------------+--

### 原始的**并行推理**模式（不使用kv缓存）
- 高显存占用
- 速度会比串行模型快->空间换时间

In [ ]:
start = time.time()
response = qwen_answer_decoding.answer_batch_decode(prompt, temperature, answer_list, geometric_mean=True)
end = time.time()
print(f'累计耗时{round(end-start,3)}秒')
response

累计耗时0.865秒


{'label_prob': [('体育', 0.9851386547088623),
  ('政治', 0.00752162653952837),
  ('财经', 0.0031354809179902077),
  ('创新设计', 0.0015884586609899998),
  ('娱乐', 0.0011534788645803928),
  ('农业民生', 0.0005663850461132824),
  ('其它', 0.00029164511943235993),
  ('科技', 9.46833097259514e-05),
  ('能源', 6.927179492777213e-05),
  ('军事', 9.374917681270745e-06),
  ('汽车', 3.908046892320272e-06),
  ('环境', 9.282424571210868e-07)],
 'label_prob_normalized': [('体育', 0.19559270458575767),
  ('政治', 0.07358325094219648),
  ('财经', 0.07326121086189301),
  ('创新设计', 0.07314796176014174),
  ('娱乐', 0.07311615079369298),
  ('农业民生', 0.0730732373518727),
  ('其它', 0.07305316397360581),
  ('科技', 0.07303877670714233),
  ('能源', 0.07303692070476922),
  ('军事', 0.07303254615230761),
  ('汽车', 0.07303214689390575),
  ('环境', 0.07303192927271476)],
 'label': '体育',
 'probability': 0.9851386547088623,
 'probability_normalized': 0.19559270458575767}

### 优化1：复用**KV缓存**，后**串行推理**
- 复用文章+问题部分的KV cache，后串行推理
- 时间复杂度：o(n+1)，具体为1次问题的推理获得KV缓存 + n次目标标签的推理
- 虽然时间复杂度为o(n+1) > o(n)，但因为复用kv cache，耗时可能比原始的串行推理模式（不使用kv缓存）还有优势（长上下文时越明显）

In [ ]:
start = time.time()
temperature = 1
response = qwen_answer_decoding.answer_decode_using_KVcache(prompt, temperature, answer_list, geometric_mean=True)
end = time.time()
print(f'累计耗时{round(end-start,3)}秒')
response

累计耗时0.444秒


{'label_prob': [('体育', 0.9847301840782166),
  ('政治', 0.008519585244357586),
  ('财经', 0.0024409021716564894),
  ('创新设计', 0.001638541929423809),
  ('娱乐', 0.0011530003976076841),
  ('农业民生', 0.0005842428654432297),
  ('其它', 0.00033034017542377114),
  ('科技', 9.464404138270766e-05),
  ('能源', 7.370886305579916e-05),
  ('军事', 1.061876719177235e-05),
  ('汽车', 4.426560735737439e-06),
  ('环境', 9.876991953206016e-07)],
 'label_prob_normalized': [('体育', 0.1955224195957247),
  ('政治', 0.07366033450839457),
  ('财经', 0.07321393481556975),
  ('创新设计', 0.0731552144256997),
  ('娱乐', 0.07311970315262371),
  ('农业民生', 0.07307812759503254),
  ('其它', 0.07305957521720358),
  ('科技', 0.07304235738693793),
  ('能源', 0.07304082824816706),
  ('军事', 0.07303622024067224),
  ('汽车', 0.07303576798671797),
  ('环境', 0.07303551682725623)],
 'label': '体育',
 'probability': 0.9847301840782166,
 'probability_normalized': 0.1955224195957247}

### 优化2：复用**KV缓存**，后**并行推理**
- 复用KV缓存 -> 后并行推理
- 时间复杂度：o(2)，即1次问题的推理获得KV缓存 + 1次目标标签的推理
- 但会增强一部分空间复杂度（显存增加），但对比不用kv缓存的并行推理模型，显存增加量小很多（因为expand操作，扩展后的对象与原对象共享内存）

In [ ]:
start = time.time()
response = qwen_answer_decoding.answer_batch_decode_using_KVcache(prompt, temperature, answer_list, geometric_mean=True)
end = time.time()
print(f'累计耗时{round(end-start,3)}秒')
response

累计耗时0.122秒


{'label_prob': [('体育', 0.9843473434448242),
  ('政治', 0.008516272529959679),
  ('财经', 0.0027648292016237974),
  ('创新设计', 0.0016382243484258652),
  ('娱乐', 0.0011525520822033286),
  ('农业民生', 0.0005661575123667717),
  ('其它', 0.0003302117402199656),
  ('科技', 9.460724686505273e-05),
  ('能源', 7.843215280445293e-05),
  ('军事', 9.971529834729154e-06),
  ('汽车', 4.156751856498886e-06),
  ('环境', 9.274968419958896e-07)],
 'label_prob_normalized': [('体育', 0.19545783760291444),
  ('政治', 0.073663956370078),
  ('财经', 0.07324149833593913),
  ('创新设计', 0.07315903057147283),
  ('娱乐', 0.07312350788619032),
  ('农业民生', 0.07308064122785463),
  ('其它', 0.07306340019358569),
  ('科技', 0.07304618815589663),
  ('能源', 0.07304500663648805),
  ('军事', 0.07304000610100073),
  ('汽车', 0.07303958139081652),
  ('环境', 0.0730393455277629)],
 'label': '体育',
 'probability': 0.9843473434448242,
 'probability_normalized': 0.19545783760291444}

### 优化4： **早停法**
- 时间复杂度o(1)
- 空间复杂度o(1) ，batch_size=1
- 最极致的加速方式，但会存在推理结果和原始方式有些微偏差的情况（误差允许范围内）

In [ ]:
start = time.time()
response = qwen_answer_decoding.answer_decode_early_stopping_method(prompt, temperature, answer_list)
end = time.time()
print(f'累计耗时{round(end-start,3)}秒')
response

累计耗时0.097秒


{'label_prob': [('体育', 0.9866604804992676),
  ('政治', 0.006648065987974405),
  ('财经', 0.0027713263407349586),
  ('娱乐', 0.0010195139329880476),
  ('其它', 0.00029209564672783017),
  ('科技', 8.36867984617129e-05),
  ('能源', 6.937879516044632e-05),
  ('军事', 8.286115189548582e-06),
  ('汽车', 3.914083663403289e-06),
  ('创新设计', 2.374011728534242e-06),
  ('环境', 9.296763892052695e-07),
  ('农业民生', 3.0182172849890776e-07)],
 'label_prob_normalized': [('体育', 0.1958829413102847),
  ('政治', 0.07351612927692484),
  ('财经', 0.07323167811165436),
  ('娱乐', 0.07310350225210814),
  ('其它', 0.07305034476398872),
  ('科技', 0.07303512201210419),
  ('能源', 0.07303407703281314),
  ('军事', 0.07302961532160834),
  ('汽车', 0.07302929603452579),
  ('创新设计', 0.07302918356424314),
  ('环境', 0.07302907808568869),
  ('农业民生', 0.07302903223405605)],
 'label': '体育',
 'probability': 0.9866604804992676,
 'probability_normalized': 0.1958829413102847}

### 优化5：**早停法** + **kv缓存利用法** + **并行推理法** 结合在一块，应对**多任务并行推理**的应用场景（multi-task parrellel inference）
- 追求高并发 极致推理速度适合使用
- 时间复杂度o(2)
- 空间复杂度o(n) n是任务的数量，但因为复用KV缓存，显存增加量还好
- KV缓存具体为重复利用了文章部分的KV缓存，问题(task)部分并不复用

In [ ]:
question = ['假设你是一个专业的文本分类器，请根据上文判断这篇文章最适合被划归到以下哪个类目结果，请直接输出类目结果，不要解释：\n财经，政治，娱乐，体育，军事，科技，农业民生，能源，汽车，创新设计，环境，其它',
       '这篇新闻提到的联赛最可能来源于哪个国家：\n英国，中国，法国，日本，美国，印度\n请直接回复最可能的语种，不要解释',
       '这篇新闻是否提到了俄乌军事冲突，请直接回复是或否，不要解释',
       '这篇新闻是否是明显的广告文案，请直接回复是或否，不要解释',
       '假设你是一个专业的文本分类器，请根据上文判断这篇文章最适合被划归到以下哪个类目结果，请直接输出类目结果，不要解释：\n财经，政治，娱乐，体育，军事，科技，能源，汽车，创新设计，环境，其它', #稍有改动
       '这篇新闻提到的联赛最可能来源于哪个国家：\n英国，中国，法国，日本，美国\n请直接回复最可能的语种，不要解释', #稍有改动
       '这篇新闻是否提到了普京，请直接回复是或否，不要解释',
       '这篇新闻是否是明显的色情文章，请直接回复是或否，不要解释']

answer_label = [['财经', '政治', '娱乐', '体育', '军事', '科技', '农业民生', '能源', '汽车', '创新设计', '环境', '其它'],
                ['英国', '中国', '法国', '日本', '美国', '印度'],
                ['是', '否'],
                ['是', '否'],
                ['财经', '政治', '娱乐', '体育', '军事', '科技', '能源', '汽车', '创新设计', '环境', '其它'],
                ['英国', '中国', '法国', '日本', '美国'],
                ['是', '否'],
                ['是', '否']
         ]

task = {k:v for k,v in zip(question, answer_label)}

In [ ]:
start = time.time()
temperature = 1
response = qwen_answer_decoding.MuitiTask_batch_decode_using_KVcache_and_EarlyStop(task, article, temperature)
end = time.time()
print(f'累计耗时{round(end-start,3)}秒')
response

累计耗时0.151秒


{'假设你是一个专业的文本分类器，请根据上文判断这篇文章最适合被划归到以下哪个类目结果，请直接输出类目结果，不要解释：\n财经，政治，娱乐，体育，军事，科技，农业民生，能源，汽车，创新设计，环境，其它': {'label_prob': [('体育',
    0.9841493964195251),
   ('政治', 0.008514560759067535),
   ('财经', 0.002764273202046752),
   ('娱乐', 0.001152320415712893),
   ('其它', 0.000374103692593053),
   ('科技', 8.885741408448666e-05),
   ('能源', 6.920222949702293e-05),
   ('军事', 1.1296952834527474e-05),
   ('汽车', 3.904122422682121e-06),
   ('创新设计', 2.5206907139363466e-06),
   ('环境', 9.273103387386072e-07),
   ('农业民生', 3.2046989417722216e-07)],
  'label_prob_normalized': [('体育', 0.19545777121155095),
   ('政治', 0.07367838822730367),
   ('财经', 0.07325593209445137),
   ('娱乐', 0.07313794211328437),
   ('其它', 0.07308104708483994),
   ('科技', 0.07306020396098255),
   ('能源', 0.07305876796330014),
   ('军事', 0.07305453759761005),
   ('汽车', 0.07305399751979912),
   ('创新设计', 0.07305389645465242),
   ('环境', 0.07305378005210021),
   ('农业民生', 0.07305373572012529)],
  'label': '体育',
  'probability': 0.9841493964195251,
  '

In [ ]:
!nvidia-smi

Fri Nov  8 16:14:25 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.161.08             Driver Version: 535.161.08   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A30                     On  | 00000000:CE:00.0 Off |                   On |
| N/A   31C    P0              54W / 165W |                  N/A |     N/A      Default |
|                                         |                      |              Enabled |
+-----------------------------------------+----------------------+--